<a href="https://colab.research.google.com/github/tsakailab/cisexpkit/blob/master/Experiment/colab/pc_octahedron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ポイントクラウドを用いた法線の計算

### [ポイントクラウドが保存されたファイル](https://github.com/tsakailab/cisexpkit/raw/master/Experiment/colab/pc_octahedron.zip)をダウンロードします．
能動的ステレオカメラRealsense SR300で取得したカラー画像と深度画像および逆透視変換したポイントクラウドのファイルが含まれています．

In [0]:
import zipfile
import os
zipURL = "https://github.com/tsakailab/cisexpkit/raw/master/Experiment/colab/pc_octahedron.zip"
!wget $zipURL --no-check-certificate --show-progress -q -O "/tmp/pc_octahedron.zip"
with zipfile.ZipFile("/tmp/pc_octahedron.zip", 'r') as f:
    f.extractall("/tmp")
!ls /tmp/

## Open3Dをインストールして，ポイントクラウドが保存されたply形式のファイルを読み込みます．

Q16: [Open3D](http://www.open3d.org/)とは何か．

Q17: [ply形式](http://paulbourke.net/dataformats/ply/)とは何か．
> ヒント：ダウンロードしたplyファイルはASCIIで保存してあるので，メモ帳などで開けます．[MeshLab](http://www.meshlab.net/)などのソフトウェアでも可視化できます．

In [0]:
!pip install -q open3d
import open3d as o3d

##カラー画像とポイントクラウドを表示します．
カラー画像の中央には正八面体があります．

In [0]:
import numpy as np
from PIL import Image
img = np.asarray(Image.open("/tmp/color00pc.png"))

%matplotlib inline
import matplotlib.pyplot as plt
plt.imshow(img)
#fig = px.imshow(img);fig.show() # to display x,y,r,g,b at mouse position

In [0]:
pcd = o3d.io.read_point_cloud("/tmp/xyzrgb0"+str(int(np.random.choice(6,1)[0]))+"pc.ply")

nd = len(pcd.points)
n = 30000
p = np.random.choice(nd, min(n,nd), replace=False)
print("%d out of %d points are displayed." % (n, nd))

import plotly.graph_objs  as go
xyz = np.asarray(pcd.points)[p,:]
rgb = np.asarray(pcd.colors)[p,:] * 1.5 # brighter

trace = go.Scatter3d(x=xyz[:,0], y=xyz[:,1], z=xyz[:,2], mode='markers',
                     marker=dict(size=2, 
                                color=['rgb({},{},{})'.format(r,g,b) for r,g,b in zip(rgb[:,0], rgb[:,1], rgb[:,2])],
                                opacity=0.5))

layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=[trace], layout=layout)
camera = dict(up=dict(x=0, y=0, z=1), center=dict(x=0, y=-0.4, z=0), eye=dict(x=0, y=0.8, z=-2))
fig.update_layout(scene_camera=camera)
fig.show()

## 法線ベクトルと，なす角を計測しましょう．
1. 正八面体から面をふたつ選んでください．
2. 選んだ面にある点の3次元座標を記録してください．ひとつの面から異なる3点の座標を記録します．
3. 記録した3点の座標から，その面の単位法線ベクトル（法線の向きを表す，大きさ1の3次元ベクトル）を算出してください．
4. 選んだ2面の法線ベクトルのなす間を計測してください．

> 可視化したポイントクラウドの点にマウスを合わせると，その点の3次元座標(X,Y,Z)が表示されます．

> 正八面体の面から点の座標を読み取りやすいように，マウス操作による視点の操作や，scatter_3dの使い方を工夫しましょう．もしポイントクラウドが再描画されなくなった場合は，ブラウザでページを開きなおしてください．

Q18: 計測した法線ベクトルのなす角を$\theta$とする．精密な正八面体を仮定すると，$\cos\theta$の真値はいくらか．

Q19: 計測を反復し，平均やばらつきなどで統計的に評価すべきである．計測した$\cos\theta$または$\theta$の[正確度(accuracy)と精度(precision)](https://en.wikipedia.org/wiki/Accuracy_and_precision)を実験的に示せ．

Q20: この3次元計測では，画像の取得からなす角の推定までの過程において，どのような原因による誤差が考えられるか．正確度または精度を損なう誤差の原因をそれぞれひとつ以上指摘し，改善方法を提案せよ．ただし，正八面体は精密に作られているものとする．